In [27]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from surprise import Dataset, Reader, SVD
# from surprise.model_selection import train_test_split # Não usado no treino final, mas bom para avaliação
import random
from io import StringIO # Para usar o CSV embutido

# Configuração para exibição do Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Constantes de Configuração
USER_DEFAULT_LOCATION = (-15.7749, -47.9294) # Brasília
DEFAULT_MAX_DISTANCE_KM = 30
NUM_CF_PRODUCT_RECOMMENDATIONS = 10 
NUM_FINAL_RECOMMENDATIONS_TO_SHOW = 5
CATEGORY_BONUS_SCORE_CONFIG = 15 

def calculate_distance_km(coord1_lat, coord1_lon, coord2_lat, coord2_lon):
    if pd.isna(coord1_lat) or pd.isna(coord1_lon) or pd.isna(coord2_lat) or pd.isna(coord2_lon):
        return float('inf')
    try:
        return geodesic((coord1_lat, coord1_lon), (coord2_lat, coord2_lon)).km
    except ValueError: # Caso as coordenadas sejam inválidas para geodesic
        return float('inf')

print("Passo 0: Importações e Configurações concluídas.")

Passo 0: Importações e Configurações concluídas.


In [28]:
# --- Carregar e Pré-processar Dados das Cooperativas ---
try:
    cooperativas_df_raw = pd.read_csv('../data/produtos_cooperativas.csv')
except FileNotFoundError:
    print("ERRO: Arquivo 'produtos_cooperativas.csv' não encontrado. Verifique o caminho.")
    exit()

# Identificar colunas de produtos (da 5ª coluna em diante)

# Identificar colunas de produtos (da 5ª coluna em diante)
product_columns = cooperativas_df_raw.columns[4:]

# Transformar para formato longo
cooperativas_produtos_list = []
for idx, row in cooperativas_df_raw.iterrows():
    coop_name = row['Nome']
    coop_region = row['Região']
    coop_lat = row['Latitude']
    coop_lon = row['Longitude']
    for product_name in product_columns:
        if row[product_name] == 'V': # 'V' indica que o produto está disponível
            cooperativas_produtos_list.append({
                'CooperativeName': coop_name,
                'Region': coop_region,
                'Latitude': coop_lat,
                'Longitude': coop_lon,
                'ProductName': product_name
            })
cooperativas_produtos_long_df = pd.DataFrame(cooperativas_produtos_list)

# Lista de todos os produtos únicos disponíveis no sistema
ALL_AVAILABLE_PRODUCTS = sorted(list(cooperativas_produtos_long_df['ProductName'].unique()))
# Lista de todas as cooperativas únicas
ALL_COOPERATIVES = sorted(list(cooperativas_produtos_long_df['CooperativeName'].unique()))

# Definir Categorias de Produtos e Mapeamento
PRODUCT_TO_CATEGORY_MAP = {
    'Abacate': 'Fruta', 'Atemóia': 'Fruta', 'Banana': 'Fruta', 'Cajamanga': 'Fruta',
    'Coco': 'Fruta', 'Goiaba': 'Fruta', 'Graviola': 'Fruta', 'Lichia': 'Fruta',
    'Limão': 'Fruta', 'Mamão': 'Fruta', 'Manga': 'Fruta', 'Maracujá': 'Fruta',
    'Pitaia': 'Fruta', 'Tangerina': 'Fruta', 'Uva': 'Fruta', 'Morango': 'Fruta',
    'Abóbora': 'Legume/Verdura', 'Abóbora italiana': 'Legume/Verdura',
    'Abóbora japonesa - Tetsukabuto': 'Legume/Verdura', 'Abóbora menina': 'Legume/Verdura',
    'Agrião': 'Folhosa', 'Alface': 'Folhosa', 'Berinjela': 'Legume/Verdura',
    'Beterraba': 'Raiz/Tubérculo', 'Brócolis - Cabeça Única': 'Legume/Verdura',
    'Brócolis - Ramoso': 'Legume/Verdura', 'Cebola': 'Tempero/Bulbo',
    'Cebolinha': 'Tempero/Bulbo', 'Cenoura': 'Raiz/Tubérculo', 'Chuchu': 'Legume/Verdura',
    'Coentro': 'Tempero/Bulbo', 'Couve': 'Folhosa', 'Couve-flor': 'Legume/Verdura',
    'Gengibre': 'Raiz/Tubérculo', 'Jiló': 'Legume/Verdura', 'Mandioca': 'Raiz/Tubérculo',
    'Milho doce': 'Grão/Cereal', 'Milho-verde': 'Grão/Cereal', 'Pepino': 'Legume/Verdura',
    'Pimentão': 'Legume/Verdura', 'Quiabo': 'Legume/Verdura', 'Repolho': 'Folhosa',
    'Tomate': 'Fruta', # Botanicamente é fruta
    'Alho': 'Tempero/Bulbo', 'Batata': 'Raiz/Tubérculo', 'Batata-doce': 'Raiz/Tubérculo'
}
# Garantir que todos os produtos estejam mapeados
for prod in ALL_AVAILABLE_PRODUCTS:
    if prod not in PRODUCT_TO_CATEGORY_MAP:
        PRODUCT_TO_CATEGORY_MAP[prod] = 'Outros' # Categoria padrão

ALL_PRODUCT_CATEGORIES = sorted(list(set(PRODUCT_TO_CATEGORY_MAP.values())))

print(f"\nPasso 1: Dados das Cooperativas Processados.")
print(f"Total de produtos únicos: {len(ALL_AVAILABLE_PRODUCTS)}")
print(f"Total de cooperativas únicas: {len(ALL_COOPERATIVES)}")
print(f"Total de categorias de produtos definidas: {len(ALL_PRODUCT_CATEGORIES)}")
# print(f"Categorias: {ALL_PRODUCT_CATEGORIES}")
# print("Amostra de cooperativas_produtos_long_df:")
# print(cooperativas_produtos_long_df.head())


Passo 1: Dados das Cooperativas Processados.
Total de produtos únicos: 46
Total de cooperativas únicas: 17
Total de categorias de produtos definidas: 6


In [29]:
# --- Gerar Dados Sintéticos de Avaliações de Usuários ---
NUM_USERS_SIMULATED = 250
NUM_RATINGS_TARGET = 6000

ratings_list_simulated = []
rated_pairs = set() 

while len(ratings_list_simulated) < NUM_RATINGS_TARGET:
    user_id = random.randint(1, NUM_USERS_SIMULATED)
    # Garantir que o produto escolhido esteja na lista de TODOS os produtos disponíveis
    if not ALL_AVAILABLE_PRODUCTS:
        print("ERRO: ALL_AVAILABLE_PRODUCTS está vazia. Verifique o Passo 1.")
        break
    product_id = random.choice(ALL_AVAILABLE_PRODUCTS)
    
    if (user_id, product_id) not in rated_pairs:
        rating = random.randint(1, 5) # Escala de 1 a 5
        ratings_list_simulated.append({'user_id': user_id, 'item_id': product_id, 'rating': rating})
        rated_pairs.add((user_id, product_id))

ratings_df = pd.DataFrame(ratings_list_simulated)

print(f"\nPasso 2: Dados Sintéticos de Avaliações Gerados.")
if not ratings_df.empty:
    print(f"Total de avaliações geradas: {len(ratings_df)}")
    # print("Amostra de ratings_df:")
    # print(ratings_df.head())
    if len(ratings_df) < 5000:
        print("AVISO: Menos de 5000 avaliações geradas. Considere aumentar NUM_USERS_SIMULATED ou NUM_RATINGS_TARGET.")
else:
    print("ERRO: Nenhuma avaliação foi gerada. Verifique a lógica de geração ou as listas de produtos/usuários.")


Passo 2: Dados Sintéticos de Avaliações Gerados.
Total de avaliações geradas: 6000


In [30]:
# --- Gerar Dados Sintéticos de Preferências Explícitas dos Usuários (com Categorias) ---
if ratings_df.empty:
    print("AVISO: ratings_df está vazio. Pulando geração de preferências de usuário, pois depende dos user_ids das avaliações.")
    user_preferences_df = pd.DataFrame(columns=['user_id', 'preferred_cooperative_name', 'preferred_product_categories', 'max_distance_preference_km'])
else:
    user_ids_for_prefs = ratings_df['user_id'].unique()
    user_preferences_list_simulated = []

    MAX_PREFERRED_CATEGORIES = 2 

    for uid in user_ids_for_prefs:
        prefs = {'user_id': uid}
        
        if random.random() < 0.2 and ALL_COOPERATIVES: # Chance de ter cooperativa preferida
            prefs['preferred_cooperative_name'] = random.choice(ALL_COOPERATIVES)
        else:
            prefs['preferred_cooperative_name'] = np.nan

        # Chance de ter categorias de produtos preferidas
        if random.random() < 0.6 and ALL_PRODUCT_CATEGORIES:
            num_cats_to_pick = random.randint(1, min(MAX_PREFERRED_CATEGORIES, len(ALL_PRODUCT_CATEGORIES)))
            prefs['preferred_product_categories'] = random.sample(ALL_PRODUCT_CATEGORIES, num_cats_to_pick)
        else:
            prefs['preferred_product_categories'] = [] 

        if random.random() < 0.4: # Chance de ter preferência de distância
            prefs['max_distance_preference_km'] = random.choice([10.0, 15.0, 25.0, 40.0])
        else:
            prefs['max_distance_preference_km'] = np.nan
            
        user_preferences_list_simulated.append(prefs)

    user_preferences_df = pd.DataFrame(user_preferences_list_simulated)

print(f"\nPasso 3: Dados Sintéticos de Preferências Explícitas Gerados (com Categorias).")
print(f"Total de perfis de preferência de usuários: {len(user_preferences_df)}")
# print("Amostra de user_preferences_df:")
# if not user_preferences_df.empty:
#     print(user_preferences_df.head())
#     user_with_cat_prefs = user_preferences_df[user_preferences_df['preferred_product_categories'].apply(lambda x: isinstance(x, list) and len(x) > 0)]
#     if not user_with_cat_prefs.empty:
#         print("\nExemplo de usuário com categorias preferidas:")
#         print(user_with_cat_prefs.head(1))


Passo 3: Dados Sintéticos de Preferências Explícitas Gerados (com Categorias).
Total de perfis de preferência de usuários: 250


In [31]:
# --- Treinar o Modelo de Filtro Colaborativo (SVD) ---
svd_model = None # Inicializar
if ratings_df.empty:
    print("AVISO: ratings_df está vazio. Não é possível treinar o modelo SVD.")
else:
    reader = Reader(rating_scale=(1, 5))
    data_surprise = Dataset.load_from_df(ratings_df[['user_id', 'item_id', 'rating']], reader)
    trainset_surprise = data_surprise.build_full_trainset()

    svd_model = SVD(n_factors=50, n_epochs=20, random_state=42, verbose=False)
    print("\nTreinando o modelo SVD...")
    svd_model.fit(trainset_surprise)
    print("Passo 4: Modelo SVD treinado com sucesso.")

# --- Função para obter recomendações de PRODUTOS do SVD ---
def get_cf_product_recommendations(user_id, model, num_recs=10):
    if model is None:
        # print(f"AVISO: Modelo SVD não treinado. Retornando lista vazia para user {user_id}.")
        return []
    try:
        user_inner_id = model.trainset.to_inner_uid(user_id)
    except ValueError:
        # print(f"Usuário {user_id} não encontrado no modelo. Retornando lista vazia.")
        return []
        
    rated_item_inner_ids = [item_id for (item_id, _) in model.trainset.ur[user_inner_id]]
    all_item_inner_ids = list(model.trainset.all_items())
    items_to_predict_inner_ids = np.setdiff1d(all_item_inner_ids, rated_item_inner_ids, assume_unique=True)

    predictions = []
    for item_inner_id in items_to_predict_inner_ids:
        raw_item_id = model.trainset.to_raw_iid(item_inner_id)
        pred = model.predict(uid=user_id, iid=raw_item_id, verbose=False)
        predictions.append(pred)
    
    predictions.sort(key=lambda x: x.est, reverse=True)
    recommended_products_with_scores = [(pred.iid, pred.est) for pred in predictions[:num_recs]]
    return recommended_products_with_scores

# Teste rápido da função de recomendação de produtos (se o modelo foi treinado)
if svd_model and not ratings_df.empty:
    test_user_id_for_cf = ratings_df['user_id'].unique()[0] # Pegar o primeiro usuário do df
    cf_recs_test = get_cf_product_recommendations(test_user_id_for_cf, svd_model, num_recs=5)
    print(f"\nRecomendações de PRODUTOS do CF para User {test_user_id_for_cf}: {cf_recs_test}")
else:
    print("\nAVISO: Teste de CF não executado pois o modelo SVD não foi treinado ou não há dados de avaliação.")


Treinando o modelo SVD...
Passo 4: Modelo SVD treinado com sucesso.

Recomendações de PRODUTOS do CF para User 86: [('Pepino', 3.432541040668577), ('Couve-flor', 3.4015230413765787), ('Cenoura', 3.2999785052730206), ('Mamão', 3.269829516090366), ('Cajamanga', 3.2593917461137507)]


In [32]:
# --- Função de Recomendação Aprimorada (v3 - Com Preferências de Categoria) ---
def get_final_cooperative_recommendations_v3(
    target_user_id,
    user_current_location,
    cf_model,                             # Modelo SVD treinado
    all_cooperatives_products_info,       # DataFrame cooperativas_produtos_long_df
    user_explicit_preferences,            # DataFrame user_preferences_df
    product_to_category_map,              # Dicionário {product_name: category_name}
    current_search_intent_product=None,
    num_initial_cf_recs=NUM_CF_PRODUCT_RECOMMENDATIONS,
    default_max_dist=DEFAULT_MAX_DISTANCE_KM,
    num_final_recs_to_show=NUM_FINAL_RECOMMENDATIONS_TO_SHOW,
    category_bonus_score=CATEGORY_BONUS_SCORE_CONFIG 
):
    print(f"\n--- Gerando Recomendações Aprimoradas (v3) para Usuário: {target_user_id} ---")
    if current_search_intent_product:
        print(f"Intenção de Busca Atual: {current_search_intent_product.upper()}")
    print(f"Localização do Usuário: {user_current_location}")

    # 1. Obter preferências explícitas do usuário
    user_prefs_row = user_explicit_preferences[user_explicit_preferences['user_id'] == target_user_id]
    
    preferred_coop_name = None
    user_max_dist = default_max_dist
    user_preferred_categories = []

    if not user_prefs_row.empty:
        prefs = user_prefs_row.iloc[0]
        preferred_coop_name = prefs.get('preferred_cooperative_name') if pd.notna(prefs.get('preferred_cooperative_name')) else None
        if pd.notna(prefs.get('max_distance_preference_km')):
            user_max_dist = prefs['max_distance_preference_km']
        # Garante que 'preferred_product_categories' seja sempre uma lista e não NaN
        if 'preferred_product_categories' in prefs and isinstance(prefs['preferred_product_categories'], list):
             user_preferred_categories = prefs['preferred_product_categories']
        elif pd.isna(prefs.get('preferred_product_categories')): # Trata caso seja NaN
             user_preferred_categories = []


    print(f"Preferências do Usuário: PrefCoop='{preferred_coop_name}', MaxDist={user_max_dist}km, PrefCatgs={user_preferred_categories}")

    # 2. Construir a lista de produtos para buscar
    product_candidates = {} # {prod_name: score_ou_prioridade}
    
    if cf_model is None: # Se o modelo CF não existe (ex: não foi treinado)
        print("AVISO: Modelo CF não disponível. Recomendações serão baseadas apenas na intenção de busca, se houver.")
        if current_search_intent_product:
            product_candidates[current_search_intent_product] = 5.0 # Score máximo para intenção
    else: # Modelo CF existe
        if current_search_intent_product:
            try: # Tentar obter score do CF para a intenção
                pred_intent_product = cf_model.predict(uid=target_user_id, iid=current_search_intent_product, verbose=False)
                product_candidates[current_search_intent_product] = pred_intent_product.est
                print(f"Score CF para intenção '{current_search_intent_product}': {pred_intent_product.est:.2f}")
            except Exception: # Se usuário/item não no modelo
                product_candidates[current_search_intent_product] = 5.0 
                print(f"Score CF para intenção '{current_search_intent_product}' não disponível (usuário/item novo?), atribuído 5.0.")

        # 3. Obter recomendações do Filtro Colaborativo para complementar
        cf_product_recs_with_scores = get_cf_product_recommendations(target_user_id, cf_model, num_recs=num_initial_cf_recs * 2)
        for prod_name, score in cf_product_recs_with_scores:
            if prod_name == current_search_intent_product: continue
            if prod_name not in product_candidates: product_candidates[prod_name] = score
    
    if not product_candidates:
        print("INFO: Nenhum produto candidato para buscar (nem intenção, nem CF).")
        return pd.DataFrame()

    # Ordenar os candidatos por score e pegar os N melhores
    sorted_product_candidates = sorted(product_candidates.items(), key=lambda item: item[1], reverse=True)
    
    final_product_names_to_search = []
    if current_search_intent_product and current_search_intent_product in product_candidates: # Garantir que a intenção esteja na lista, se existir
        final_product_names_to_search.append(current_search_intent_product)
    
    for prod, score in sorted_product_candidates:
        if len(final_product_names_to_search) >= num_initial_cf_recs: break
        if prod not in final_product_names_to_search: final_product_names_to_search.append(prod)
            
    if not final_product_names_to_search:
        print("INFO: Nenhum produto final para buscar em cooperativas.")
        return pd.DataFrame()
        
    print(f"Produtos finais para buscar em cooperativas: {final_product_names_to_search}")

    # 5. Encontrar cooperativas
    coops_with_target_products_df = all_cooperatives_products_info[
        all_cooperatives_products_info['ProductName'].isin(final_product_names_to_search)
    ].copy()

    if coops_with_target_products_df.empty:
        print(f"INFO: Nenhuma cooperativa encontrada que venda os produtos: {final_product_names_to_search}")
        return pd.DataFrame()
        
    # 6. Calcular distância
    coops_with_target_products_df['Distance_km'] = coops_with_target_products_df.apply(
        lambda row: calculate_distance_km(user_current_location[0], user_current_location[1], 
                                          row['Latitude'], row['Longitude']), axis=1
    )
    
    # 7. Filtrar por distância
    nearby_cooperatives_df = coops_with_target_products_df[
        coops_with_target_products_df['Distance_km'] <= user_max_dist
    ].copy()

    if nearby_cooperatives_df.empty:
        print(f"INFO: Nenhuma cooperativa encontrada dentro de {user_max_dist} km que venda os produtos buscados.")
        return pd.DataFrame()

    # 8. Pontuação e Re-ranqueamento
    nearby_cooperatives_df['CF_Score'] = nearby_cooperatives_df['ProductName'].map(product_candidates).fillna(0)
    nearby_cooperatives_df['RelevanceScore'] = 0.0
    nearby_cooperatives_df['RelevanceScore'] += nearby_cooperatives_df['Distance_km'] * 0.1 
    nearby_cooperatives_df['RelevanceScore'] -= nearby_cooperatives_df['CF_Score']     

    if preferred_coop_name:
        nearby_cooperatives_df.loc[nearby_cooperatives_df['CooperativeName'] == preferred_coop_name, 'RelevanceScore'] -= 50 
    
    if current_search_intent_product:
         nearby_cooperatives_df.loc[nearby_cooperatives_df['ProductName'] == current_search_intent_product, 'RelevanceScore'] -= 100

    if user_preferred_categories:
        for index, row_rec in nearby_cooperatives_df.iterrows():
            product_cat = product_to_category_map.get(row_rec['ProductName'])
            if product_cat and product_cat in user_preferred_categories:
                if row_rec['ProductName'] != current_search_intent_product: # Evitar duplo bônus se intenção já é da categoria
                    nearby_cooperatives_df.loc[index, 'RelevanceScore'] -= category_bonus_score

    final_sorted_recommendations = nearby_cooperatives_df.sort_values(
        by=['RelevanceScore', 'Distance_km']
    ).reset_index(drop=True)
    
    result_df = final_sorted_recommendations[[
        'CooperativeName', 'ProductName', 'Distance_km', 'Region', 'CF_Score', 'RelevanceScore'
    ]]
    
    print(f"Passo 5 (v3): Recomendações Finais Geradas.")
    return result_df.head(num_final_recs_to_show)

print("Passo 5: Função de Recomendação Aprimorada (v3 - com categorias) definida.")

Passo 5: Função de Recomendação Aprimorada (v3 - com categorias) definida.


In [ ]:
# --- Passo X: Criar e Testar Usuário Artificial com Intenção e Preferências de Categoria ---

if svd_model is None or ratings_df.empty or user_preferences_df.empty or cooperativas_produtos_long_df.empty:
    print("\nAVISO: Pelo menos um dos DataFrames necessários (ratings, user_preferences, cooperativas) está vazio ou o modelo SVD não foi treinado. Teste pulado.")
else:
    # 1. Definir as características do usuário de teste
    test_user_id_v3 = 9999
    # test_user_current_intent_v3 = random.choice(ALL_AVAILABLE_PRODUCTS) if ALL_AVAILABLE_PRODUCTS else 'Tomate'
    # test_user_preferred_categories_v3 = random.sample(ALL_PRODUCT_CATEGORIES, k=min(2, len(ALL_PRODUCT_CATEGORIES))) if ALL_PRODUCT_CATEGORIES else ['Fruta']
    # test_user_preferred_coop_v3 = random.choice(ALL_COOPERATIVES) if ALL_COOPERATIVES else np.nan
    test_user_max_dist_v3 = 25.0
    test_user_current_intent_v3 = None
    test_user_preferred_categories_v3 = []
    test_user_preferred_coop_v3 = None
    test_user_location_v3 = USER_DEFAULT_LOCATION

    print(f"\n--- Criando/Usando Usuário de Teste com Intenção e Categorias: ID {test_user_id_v3} ---")
    print(f"Quer AGORA: '{test_user_current_intent_v3}', Pref Cats: {test_user_preferred_categories_v3}, Pref Coop: '{test_user_preferred_coop_v3}', Dist Max: {test_user_max_dist_v3}km")

    # 2. Adicionar/Modificar o usuário no user_preferences_df
    if test_user_id_v3 in user_preferences_df['user_id'].values:
        idx_to_update = user_preferences_df[user_preferences_df['user_id'] == test_user_id_v3].index
        # Para colunas que são listas, precisamos atribuir de forma especial para evitar erros de SettingWithCopy
        user_preferences_df.loc[idx_to_update, 'preferred_product_categories'] = pd.Series([test_user_preferred_categories_v3] * len(idx_to_update), index=idx_to_update)
        user_preferences_df.loc[idx_to_update, 'preferred_cooperative_name'] = test_user_preferred_coop_v3
        user_preferences_df.loc[idx_to_update, 'max_distance_preference_km'] = test_user_max_dist_v3
    else:
        new_pref_entry_v3 = pd.DataFrame([{\
            'user_id': test_user_id_v3,
            'preferred_product_categories': test_user_preferred_categories_v3,
            'preferred_cooperative_name': test_user_preferred_coop_v3,
            'max_distance_preference_km': test_user_max_dist_v3
        }])
        user_preferences_df = pd.concat([user_preferences_df, new_pref_entry_v3], ignore_index=True)
    print(f"Usuário {test_user_id_v3} adicionado/atualizado em user_preferences_df.")

    # 3. Adicionar algumas avaliações para este usuário ao ratings_df (para o modelo SVD ter dados)
    new_ratings_for_v3_user = []
    if test_user_current_intent_v3 in ALL_AVAILABLE_PRODUCTS:
        new_ratings_for_v3_user.append({'user_id': test_user_id_v3, 'item_id': test_user_current_intent_v3, 'rating': random.randint(3,5)})

    prods_in_pref_cats_v3 = [p for p, cat in PRODUCT_TO_CATEGORY_MAP.items() if cat in test_user_preferred_categories_v3 and p != test_user_current_intent_v3]
    if prods_in_pref_cats_v3:
        for prod_to_rate in random.sample(prods_in_pref_cats_v3, k=min(2, len(prods_in_pref_cats_v3))):
             new_ratings_for_v3_user.append({'user_id': test_user_id_v3, 'item_id': prod_to_rate, 'rating': random.choice([4,5])})
    
    # Adicionar mais algumas avaliações aleatórias para ter um perfil mínimo
    num_additional_random_ratings = 3
    available_prods_for_random_rating = [p for p in ALL_AVAILABLE_PRODUCTS if p not in [r['item_id'] for r in new_ratings_for_v3_user]]
    if len(available_prods_for_random_rating) >= num_additional_random_ratings :
        for prod_rand in random.sample(available_prods_for_random_rating, k=num_additional_random_ratings):
            new_ratings_for_v3_user.append({'user_id': test_user_id_v3, 'item_id': prod_rand, 'rating': random.randint(1,4)})


    current_svd_model_to_use = svd_model # Por padrão usa o modelo global
    if new_ratings_for_v3_user:
        temp_ratings_df_for_retrain = ratings_df.copy()
        # Remover avaliações antigas do usuário de teste, se houver, antes de adicionar novas
        temp_ratings_df_for_retrain = temp_ratings_df_for_retrain[temp_ratings_df_for_retrain['user_id'] != test_user_id_v3]
        new_ratings_df_v3 = pd.DataFrame(new_ratings_for_v3_user)
        temp_ratings_df_for_retrain = pd.concat([temp_ratings_df_for_retrain, new_ratings_df_v3], ignore_index=True)
        
        print(f"Novas avaliações adicionadas para o usuário {test_user_id_v3}. Re-treinando o modelo SVD temporariamente...")
        reader_retrain_v3 = Reader(rating_scale=(1, 5))
        data_surprise_retrain_v3 = Dataset.load_from_df(temp_ratings_df_for_retrain[['user_id', 'item_id', 'rating']], reader_retrain_v3)
        trainset_surprise_retrain_v3 = data_surprise_retrain_v3.build_full_trainset()
        
        svd_model_retrained_v3 = SVD(n_factors=50, n_epochs=20, random_state=42, verbose=False)
        svd_model_retrained_v3.fit(trainset_surprise_retrain_v3)
        print("Modelo SVD re-treinado temporariamente para o teste.")
        current_svd_model_to_use = svd_model_retrained_v3 # Usa o modelo re-treinado para este teste
    else:
        print("Nenhuma nova avaliação para o usuário de teste. Usando o modelo SVD globalmente treinado.")


    # 5. Executar a função de recomendação v3
    print(f"\n--- Obtendo recomendações para Usuário {test_user_id_v3} (INTENÇÃO: '{test_user_current_intent_v3}') ---")
    v3_user_recommendations = get_final_cooperative_recommendations_v3(
        target_user_id=test_user_id_v3,
        user_current_location=test_user_location_v3,
        cf_model=current_svd_model_to_use, # Usa o modelo (possivelmente re-treinado)
        all_cooperatives_products_info=cooperativas_produtos_long_df,
        user_explicit_preferences=user_preferences_df,
        product_to_category_map=PRODUCT_TO_CATEGORY_MAP,
        current_search_intent_product=test_user_current_intent_v3,
        num_initial_cf_recs=NUM_CF_PRODUCT_RECOMMENDATIONS,
        default_max_dist=DEFAULT_MAX_DISTANCE_KM,
        num_final_recs_to_show=NUM_FINAL_RECOMMENDATIONS_TO_SHOW,
        category_bonus_score=CATEGORY_BONUS_SCORE_CONFIG
    )

    # Formatar a saída
    if not v3_user_recommendations.empty:
        print(f"\n✨ === TOP {len(v3_user_recommendations)} RECOMENDAÇÕES PARA USUÁRIO {test_user_id_v3} (INTENÇÃO: '{test_user_current_intent_v3.upper()}') === ✨")
        print(f"   (Considerando Prefs: Cats={test_user_preferred_categories_v3}, Coop='{test_user_preferred_coop_v3}')")
        print("-" * 70)
        for index, rec in v3_user_recommendations.iterrows():
            rec_product_cat = PRODUCT_TO_CATEGORY_MAP.get(rec['ProductName'], 'N/A')
            cat_bonus_applied_msg = ""
            if rec_product_cat in test_user_preferred_categories_v3 and rec['ProductName'] != test_user_current_intent_v3:
                 cat_bonus_applied_msg = f"(Cat. Preferida: {rec_product_cat})"

            print(f"🛒 Produto: {rec['ProductName'].upper()} {cat_bonus_applied_msg}")
            print(f"   📍 Cooperativa: {rec['CooperativeName']}")
            print(f"   🗺️ Região: {rec['Region']}")
            print(f"   🚗 Distância: {rec['Distance_km']:.1f} km")
            # print(f"   ⭐ Score CF: {rec['CF_Score']:.2f} | ⚖️ RelevanceScore: {rec['RelevanceScore']:.2f}") # Para depuração
            print("-" * 70)
    else:
        print(f"\n🙁 INFO: Nenhuma recomendação final pôde ser gerada para o usuário {test_user_id_v3} (INTENÇÃO: '{test_user_current_intent_v3.upper()}') com os critérios atuais.")

print("\n--- Teste com usuário com intenção de busca e preferências de categoria concluído. ---")


--- Criando/Usando Usuário de Teste com Intenção e Categorias: ID 10000 ---
Quer AGORA: 'None', Pref Cats: [], Pref Coop: 'None', Dist Max: 25.0km
Usuário 10000 adicionado/atualizado em user_preferences_df.
Novas avaliações adicionadas para o usuário 10000. Re-treinando o modelo SVD temporariamente...
Modelo SVD re-treinado temporariamente para o teste.

--- Obtendo recomendações para Usuário 10000 (INTENÇÃO: 'None') ---

--- Gerando Recomendações Aprimoradas (v3) para Usuário: 10000 ---
Localização do Usuário: (-15.7749, -47.9294)
Preferências do Usuário: PrefCoop='None', MaxDist=25.0km, PrefCatgs=[]
Produtos finais para buscar em cooperativas: ['Cenoura', 'Abóbora', 'Coentro', 'Goiaba', 'Repolho', 'Beterraba', 'Graviola', 'Mamão', 'Abóbora japonesa - Tetsukabuto', 'Couve-flor']
Passo 5 (v3): Recomendações Finais Geradas.


AttributeError: 'NoneType' object has no attribute 'upper'